In [ ]:
# import libraries
import json
import math
import os
import tensorflow
from tensorflow import keras
from keras import Sequential
from glob import glob
from tqdm import tqdm
from PIL import Image
import cv2 # image processing
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # data visualization
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop
from keras.applications import DenseNet201
from keras.utils.np_utils import to_categorical
from keras.layers import  Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models, optimizers, applications
from tensorflow.keras.regularizers import l2

In [ ]:
input_dir = "E:/Fall 23/CSE438/Final Paper/Final Dataset/"
train_dir = os.path.join(input_dir, "train")
test_dir = os.path.join(input_dir, "test")
#val_dir = input_dir +"val/"

In [ ]:

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)


test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)



# Setup the training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training',

)

# Setup the test data generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle = False
)

# Setup the validation data generator
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'

)

In [ ]:
# Create DenseNet201 Model with Keras library

def build_model(backbone, lr=1e-4, dropout_rate=0.5):

    for layer in backbone.layers[:-10]:  # Fine-tune from the last 10 layers
        layer.trainable = True

    model= models.Sequential([
        backbone,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(dropout_rate),
        layers.BatchNormalization(),
        layers.Dense(4, activation='softmax', kernel_regularizer=l2(0.001))
    ])

    optimizer = optimizers.Adam(learning_rate=lr)
    metrics = ['accuracy', Precision(), Recall()]

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metrics)
    return model
input_shape = (150,150,3)
densenet_backbone = applications.DenseNet201(weights='imagenet', include_top= False, input_shape=input_shape)

model= build_model(densenet_backbone, lr=1e-4)
model.summary()


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# Fit Model



# Define Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# Define Model Checkpoint
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Define Reduce Learning Rate on Plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6, verbose=1)

# Define Learning Rate Scheduler (optional)
def lr_schedule(epoch, lr):
    if epoch > 20:
        return lr * tf.math.exp(-0.1)
    return lr

learning_rate_scheduler = LearningRateScheduler(lr_schedule, verbose=1)


# Redefine the model
model = build_model(densenet_backbone, lr=1e-4)  # Adjust the learning rate here

# Compile the model
optimizer = optimizers.Adam(learning_rate=1e-4)  # Adjust the learning rate here
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', Precision(), Recall()])

# Compile your model here (if not done before)

# Fit Model
history = model.fit(
    train_generator,
    #steps_per_epoch=100,
    epochs=50,
    validation_data=validation_generator,
    #validation_steps=50,
    #callbacks=[early_stopping, model_checkpoint, reduce_lr, learning_rate_scheduler],
    verbose=1
)


In [ ]:
# Visualize Loss and Accuracy Rates
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [ ]:
# Evaluate the model on test set
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator)

# Print evaluation metrics
print(f'Test Accuracy: {test_accuracy}')
print(f'Test Precision: {test_precision}')
print(f'Test Recall: {test_recall}')
print(f'Test Loss: {test_loss}')
from sklearn.metrics import classification_report


# Generate predictions for the test set
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels for the test set
true_labels = test_generator.classes

# Ensure the true labels and predicted labels have the same length
if predicted_classes.shape[0] != true_labels.shape[0]:
    n_seq = true_labels.shape[0] // predicted_classes.shape[0]
    true_labels = np.repeat(true_labels, n_seq)

# Generate a classification report
report = classification_report(true_labels, predicted_classes, target_names=test_generator.class_indices.keys())
print(report)


In [ ]:
# Evaluate the model on test set
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator)

# Print evaluation metrics
print(f'Test Accuracy: {test_accuracy}')
print(f'Test Precision: {test_precision}')
print(f'Test Recall: {test_recall}')


from sklearn.metrics import classification_report
import numpy as np

# Generate predictions for the test set
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels for the test set
true_labels = test_generator.classes

# Ensure the true labels and predicted labels have the same length
if predicted_classes.shape[0] != true_labels.shape[0]:
    n_seq = true_labels.shape[0] // predicted_classes.shape[0]
    true_labels = np.repeat(true_labels, n_seq)

# Generate a classification report
report = classification_report(true_labels, predicted_classes, target_names=test_generator.class_indices.keys())
print(report)



import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Get the true labels
true_labels = test_generator.classes

# Predict the labels using the model
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()